[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/tagging.ipynb)

# 将文本分类为标签

标签化是指为文档标注类别，例如：

- 情感
- 语言
- 风格（正式、非正式等）
- 涵盖的主题
- 政治倾向

![图片描述](../../static/img/tagging.png)

## 概述

标签化有以下几个组成部分：

* `function`：与[提取](/docs/tutorials/extraction)类似，标签化使用[函数](https://openai.com/blog/function-calling-and-other-api-updates)来指定模型如何标注文档
* `schema`：定义我们希望如何标注文档

## 快速开始

让我们看一个如何在 LangChain 中使用 OpenAI 工具调用进行标签化的简单示例。我们将使用 OpenAI 模型支持的[`with_structured_output`](/docs/how_to/structured_output)方法。

In [ ]:
pip install --upgrade --quiet langchain-core

我们需要加载一个[聊天模型](/docs/integrations/chat/):

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

让我们用 Pydantic 模型指定一些属性及其在 schema 中的预期类型。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
提取以下段落中的所需信息。

仅提取 'Classification' 函数中提到的属性。

段落：
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="文本的情感")
    aggressiveness: int = Field(
        description="文本的攻击性程度，范围为 1 到 10"
    )
    language: str = Field(description="文本的语言")


# LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

In [8]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

如果我们想要字典输出，只需调用 `.model_dump()`

In [10]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': 'enojado', 'aggressiveness': 8, 'language': 'es'}

正如我们在示例中看到的，它正确地解释了我们的需求。

结果可能会有所不同，例如我们可能会得到不同语言的情感（'positive'，'enojado' 等）。

我们将在下一节中看到如何控制这些结果。

## 更精细的控制

仔细的 schema 定义可以让我们更好地控制模型的输出。

具体来说，我们可以定义：

- 每个属性的可能值
- 确保模型理解属性的描述
- 必须返回的属性

让我们重新声明我们的 Pydantic 模型，以控制前面提到的每个方面，使用枚举：

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="描述语句的攻击性程度，数字越高越具攻击性",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
提取以下段落中的所需信息。

仅提取 'Classification' 函数中提到的属性。

段落：
{input}
"""
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

现在答案将以我们期望的方式受到限制！

In [12]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [13]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='enojado', aggressiveness=8, language='es')

In [14]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification(sentiment='neutral', aggressiveness=1, language='English')

[LangSmith 跟踪](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r) 让我们可以窥探其内部工作原理：

![图片描述](../../static/img/tagging_trace.png)

### 更深入的探索

* 您可以使用 [metadata tagger](/docs/integrations/document_transformers/openai_metadata_tagger) 文档转换器从 LangChain `Document` 中提取元数据。
* 这涵盖了与标签链相同的基本功能，只是应用于 LangChain `Document`。